# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AQ8.KQ94.JT.KJ52'

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1N', 'insta_score': 0.998}]

In [4]:
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': '4H', 'insta_score': 0.764, 'expected_score': 419, 'adjustment': 38},
 {'call': '3N', 'insta_score': 0.218, 'expected_score': 312, 'adjustment': 11}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AQ8.KQ9x.JT.KJxx Kxxxx.Tx.KQ9x.xx J.AJxx.Axxxx.AQ8 T9xx.8xx.8x.T9xx 0.80515 \n P-1N-P-3S-P-4H-P-4N-P-P-P (12.2)  \n P-1N-P-3S-P-3N-P-4C-P-4H-P-6N-P-P-P (12.2) ',
 'AQ8.KQ9x.JT.KJxx 9xxxx.T8xx.8x.QT K.AJxx.AQ9xx.98x JTxx.x.Kxxx.Axxx 0.30526 \n P-1N-P-3S-P-4H-P-P-P (10.9)  \n P-1N-P-3S-P-3N-P-P-P (10.0) ',
 'AQ8.KQ9x.JT.KJxx K9xxxx.Jxx.xxx.x T.AT8x.K8x.AQTxx Jxx.xx.AQ9xx.98x 0.86629 \n P-1N-P-3S-P-4H-P-P-P (11.3)  \n P-1N-P-3S-P-3N-P-P-P (9.7) ',
 'AQ8.KQ9x.JT.KJxx KTxxx.T8x.KQx.Q8 x.AJxx.A8xx.ATxx J9xx.xx.9xxx.9xx 0.98222 \n P-1N-P-3S-P-4H-P-P-P (12.4)  \n P-1N-P-3S-P-3N-P-P-P (11.8) ',
 'AQ8.KQ9x.JT.KJxx J9xxxx.xx.KQ98.A .AT8x.Axx.T98xxx KTxx.Jxx.xxxx.Qx 0.34745 \n P-1N-P-3S-P-4H-P-P-P (11.3)  \n P-1N-P-3S-P-3N-P-P-P (9.1) ',
 'AQ8.KQ9x.JT.KJxx KTxx.T8x.xxxx.xx x.Axxx.K9x.AT98x J9xxx.Jx.AQ8x.Qx 0.46020 \n P-1N-P-3S-P-4H-P-P-P (11.4)  \n P-1N-P-3S-P-3N-P-P-P (9.3) ',
 'AQ8.KQ9x.JT.KJxx 9xxx.8x.KQxxx.xx K.ATxx.A9xx.T9xx JTxxx.Jxx.8x.AQ8 0.55030 \n P-1N-P-3S-P-4H-P-P-P (11.2)  \n P-1N-P

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

AQ8.KQ9x.JT.KJxx Txxx.xx.Q98x.8xx K.T8xx.Axxx.A9xx J9xxx.AJx.Kxx.QT
AQ8.KQ9x.JT.KJxx KJT9x.xx.xx.AQxx x.Txxx.AKQx.T98x xxxx.AJ8.98xxx.x
AQ8.KQ9x.JT.KJxx KTxxx.xxx.98xx.9 .T8xx.KQxx.AQxxx J9xxx.AJ.Axx.T8x
AQ8.KQ9x.JT.KJxx KJ9xx.JTx.8xx.Ax x.A8xx.Axxx.Q8xx Txxx.xx.KQ9x.T9x
AQ8.KQ9x.JT.KJxx KJTxx.J8xx.Kx.9x .ATxx.A9xx.AQT8x 9xxxx.x.Q8xxx.xx
AQ8.KQ9x.JT.KJxx Jxxxx.8xx.Kx.Q8x .Txxx.AQ8xxx.ATx KT9xx.AJ.9xx.9xx
AQ8.KQ9x.JT.KJxx KJ9x.AT8x.Q8.T9x .Jxxx.AKxxx.AQ8x Txxxxx.x.9xxx.xx
AQ8.KQ9x.JT.KJxx T9xxxx.Ax.8xx.8x x.8xxx.AKQxx.AQx KJx.JTx.9xx.T9xx
AQ8.KQ9x.JT.KJxx J9xx.A8xx.xxx.Qx K.Jxxx.AQ9x.AT8x Txxxx.T.K8xx.9xx
AQ8.KQ9x.JT.KJxx Jxxx.T.Qxxx.T98x x.J8xx.AK9x.Axxx KT9xx.Axxx.8xx.Q
AQ8.KQ9x.JT.KJxx KJxxx.8x.Q8x.A8x x.Txxx.AKxx.Q9xx T9xx.AJx.9xxx.Tx
AQ8.KQ9x.JT.KJxx J9xxx.T8x.AKx.Tx K.Axxx.Q98xx.A9x Txxx.Jx.xxx.Q8xx
AQ8.KQ9x.JT.KJxx K9xxxx.ATx.9xx.x x.J8xx.AKx.AT9xx JTx.xx.Q8xxx.Q8x
AQ8.KQ9x.JT.KJxx KJxxxx.Jxx.K9x.x .8xxx.A8xx.A98xx T9xx.AT.Qxxx.QTx
AQ8.KQ9x.JT.KJxx 9xxx.T8xx.8x.Q8x T.AJxx.KQ9xx.A